In [ ]:
import celltk
import numpy as np
import plotly.graph_objects as go
from collections import Counter

ph = celltk.PlotHelper()

In [ ]:
arr = celltk.ExperimentArray.load('example.hdf5')

## Distribution of cells per condition

In [ ]:
cells = np.array([a[3] for a in arr.shape]).ravel()
fig = ph.histogram_plot(cells, nbins=5, colors=['dimgray'],
                        x_label='Number of cells', y_label='Number of conditions',
                        legend=False)
fig.show()

## Density of cells at each location in initial frame

In [ ]:
fig = celltk.PlotHelper().heatmap2d_plot(np.concatenate(arr['nuc', 'fitc', 'x', :, 0]).ravel(),
                                         np.concatenate(arr['nuc', 'fitc', 'y', :, 0]).ravel(),
                                         xbinsize=20, ybinsize=20, colorscale='Greys',
                                         x_label='X position', y_label='Y position')
fig.show()

## Distribution of nuclear areas for whole experiment

In [ ]:
fig = celltk.PlotHelper().histogram_plot(arr['nuc', 'tritc', 'area', :, 0], 
                                         include_kde=True, include_histogram=False,
                                         keys=arr.conditions, histnorm='probability density')
fig.show()

## Mean nuclear marker intensity in initial frame by location in field of view

In [ ]:
def illumination_bias(xarr, yarr, iarr):
    # Remove nans
    nan_mask = np.logical_or(np.isnan(xarr), np.isnan(yarr))
    xarr = xarr[~nan_mask]
    yarr = yarr[~nan_mask]
    iarr = iarr[~nan_mask]
    
    # Discretize data
    xbins = np.histogram_bin_edges(np.arange(xarr.max()), bins=39)
    ybins = np.histogram_bin_edges(np.arange(yarr.max()), bins=39)
    xdig = np.digitize(xarr, bins=xbins, right=True)
    ydig = np.digitize(yarr, bins=ybins, right=True)
    zdim = Counter(zip(xdig,ydig)).most_common()[0][1]
    
    # Make destination array
    temp = np.empty((40, 40, zdim))
    temp[:] = np.nan

    # Transfer data
    for x, y, val in zip(xdig, ydig, iarr):
        x -= 1
        y -= 1
        z = np.argmax(np.isnan(temp[x, y, :]))
        temp[x, y, z] = val
        
    return temp

# Collect data
x, y, i = zip(*arr['nuc', 'tritc', ('x', 'y', 'median_intensity'), :, 0])
illum = illumination_bias(np.concatenate(x), np.concatenate(y), np.concatenate(i))

fig = celltk.PlotHelper().heatmap_plot(np.nanmean(illum, axis=-1), 
                                       zmin=1000, zmax=3000, colorscale='Greys')
fig.show()



## Distributions of mean distance travelled per frame

In [ ]:
def distance_traveled(arr):
    return [np.sqrt(np.sum(np.diff(a, axis=-1) ** 2, axis=0)) for a in arr]

conds = [c for c in arr.conditions if 'il6' not in c and 'torin' not in c]
diff = distance_traveled(arr[conds]['nuc', 'fitc', ('x', 'y')])
fig = celltk.PlotHelper().histogram_plot([np.nanmean(d, axis=1) for d in diff], 
                                         include_histogram=False, include_kde=True, 
                                         keys=conds, histnorm='probability density',
                                         colors=['dimgray'], alpha=0.5)


conds = [c for c in arr.conditions if 'il6' in c and 'torin' not in c]
diff = distance_traveled(arr[conds]['nuc', 'fitc', ('x', 'y')])
fig = celltk.PlotHelper().histogram_plot([np.nanmean(d, axis=1) for d in diff], 
                                         include_histogram=False, include_kde=True, 
                                         keys=conds, histnorm='probability density',
                                         colors=['magenta'], figure=fig)
fig.show()


## HIF-1$\alpha$ activtion in control and mTOR-inhibited cells

In [ ]:
fig = go.Figure().set_subplots(1, 2, shared_yaxes=False, horizontal_spacing=0.05,
                               x_title='Time (hours)', y_title='Cells', 
                               subplot_titles=['Control', 'Torin1'])

fig = ph.heatmap_plot(arr['ifny_control']['nuc', 'fitc', 'median_intensity'], row=1, col=1,
                      figure=fig, zmin=1000, zmax=3000, colorscale='Greys', x_limit=[0, 60],
                      sort='cosine')
fig.update_xaxes(tickmode='array', tickvals=[0, 24, 48], ticktext=['0', '4', '8',], row=1, col=1)
fig.update_yaxes(ticks='', showticklabels=False, row=1, col=1)
fig = ph.heatmap_plot(arr['ifny_torin']['nuc', 'fitc', 'median_intensity'], row=1, col=2,
                      figure=fig, zmin=1000, zmax=3000, colorscale='Greys', x_limit=[0, 60],
                      sort='cosine')
fig.update_xaxes(tickmode='array', tickvals=[0, 24, 48], ticktext=['0', '4', '8',], row=1, col=2)
fig.update_yaxes(ticks='', showticklabels=False, row=1, col=2)